In [2]:
import os
import pandas as pd
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from getpass import getpass
import neptune.new as neptune
import neptune.new.integrations.sklearn as npt_utils

In [3]:
if "NEPTUNE_API_TOKEN" not in os.environ.keys() or os.environ["NEPTUNE_API_TOKEN"] == '':
    os.environ["NEPTUNE_API_TOKEN"] = getpass("Enter your Neptune API token: ")

In [4]:
training_df = pd.read_parquet(f"../data/loan_table_training.parquet") # load train dataset

In [5]:
params = {
    "max_depth": 10,
    "max_leaf_nodes": 10,
}

In [6]:
# start experiment in neptune

run = neptune.init_run(
    source_files=['neptune_experiment.ipynb'],
    project="zsotnich/mltest"
)

https://app.neptune.ai/zsotnich/mltest/e/MLTES-29


Info (NVML): NVML Shared Library Not Found. GPU usage metrics may not be reported. For more information, see https://docs.neptune.ai/help/nvml_error/


Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [7]:
run["model/parameters"] = params

In [8]:
classifier = tree.DecisionTreeClassifier(**params)

In [9]:
target = "loan_status"

x = training_df[
    training_df.columns.drop(target)
    .drop("event_timestamp")
    .drop("created_timestamp")
    .drop("loan_id")
    .drop("zipcode")
    .drop("dob_ssn")
]
X = x.reindex(sorted(x.columns), axis=1)
y = training_df.loc[:, target]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=1234
)

In [11]:
# log (save) train and test datasets version in neptune
pd.DataFrame(X_train, y_train).to_parquet("train_df.parquet")
pd.DataFrame(X_test, y_test).to_parquet("test_df.parquet")

run["datasets/train"].track_files("train_df.parquet")
run["datasets/test"].track_files("test_df.parquet")

In [12]:
classifier.fit(X_train[sorted(X_train)], y_train)

DecisionTreeClassifier(max_depth=10, max_leaf_nodes=10)

In [14]:
# log f1 into neptune

y_train_pred = classifier.predict_proba(X_train)
y_test_pred = classifier.predict_proba(X_test)

train_f1 = f1_score(y_train, y_train_pred.argmax(axis=1), average="macro")
test_f1 = f1_score(y_test, y_test_pred.argmax(axis=1), average="macro")

run["train/f1"] = train_f1
run["test/f1"] = test_f1

In [15]:
# log score into neptune

score = classifier.score(X_test, y_test)
run["test/score"] = score

In [16]:
# log out-of-the box summary into neptune

run["cls_summary"] = npt_utils.create_classifier_summary(
    classifier, X_train, X_test, y_train, y_test
)

/opt/homebrew/Caskroom/miniconda/base/envs/mltest/lib/python3.9/site-packages/neptune/new/attributes/attribute.py:64: NeptuneDeprecationWarning: The object you're logging will be implicitly cast to a string. We'll end support of this behavior in `neptune-client==1.0.0`. To log the object as a string, use `str(object)` instead.
  return self.assign(value, wait)


In [17]:
# finish experiment in neptune

run.stop()

Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 3 operations to synchronize with Neptune. Do not kill this process.
All 3 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/zsotnich/mltest/e/MLTES-29
